In [13]:
import pandas as pd
import numpy as np

!pwd

/home/afroroboticist/flaskProjectSpace/movieRecommender/movieRecommender


In [14]:
movie_data = pd.read_csv('static/movies_metadata.csv')
ratings = pd.read_csv('static/ratings_small.csv')
links = pd.read_csv('static/links_small.csv')
display(movie_data.head())


/home/afroroboticist/anaconda3/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3146: DtypeWarning: Columns (10) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


,adult,belongs_to_collection,budget,genres,homepage,id,imdb_id,original_language,original_title,overview,...,release_date,revenue,runtime,spoken_languages,status,tagline,title,video,vote_average,vote_count
0,False,"{'id': 10194, 'name': 'Toy Story Collection', ...",30000000,"[{'id': 16, 'name': 'Animation'}, {'id': 35, '...",http://toystory.disney.com/toy-story,862,tt0114709,en,Toy Story,"Led by Woody, Andy's toys live happily in his ...",...,1995-10-30,373554033.0,81.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,NaN,Toy Story,False,7.7,5415.0
1,False,NaN,65000000,"[{'id': 12, 'name': 'Adventure'}, {'id': 14, '...",NaN,8844,tt0113497,en,Jumanji,When siblings Judy and Peter discover an encha...,...,1995-12-15,262797249.0,104.0,"[{'iso_639_1': 'en', 'name': 'English'}, {'iso...",Released,Roll the dice and unleash the excitement!,Jumanji,False,6.9,2413.0
2,False,"{'id': 119050, 'name': 'Grumpy Old Men Collect...",0,"[{'id': 10749, 'name': 'Romance'}, {'id': 35, ...",NaN,15602,tt0113228,en,Grumpier Old Men,A family wedding reignites the ancient feud be...,...,1995-12-22,0.0,101.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Still Yelling. Still Fighting. Still Ready for...,Grumpier Old Men,False,6.5,92.0
3,False,NaN,16000000,"[{'id': 35, 'name': 'Comedy'}, {'id': 18, 'nam...",NaN,31357,tt0114885,en,Waiting to Exhale,"Cheated on, mistreated and stepped on, the wom...",...,1995-12-22,81452156.0,127.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Friends are the people who let you be yourself...,Waiting to Exhale,False,6.1,34.0
4,False,"{'id': 96871, 'name': 'Father of the Bride Col...",0,"[{'id': 35, 'name': 'Comedy'}]",NaN,11862,tt0113041,en,Father of the Bride Part II,Just when George Banks has recovered from his ...,...,1995-02-10,76578911.0,106.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Just When His World Is Back To Normal... He's ...,Father of the Bride Part II,False,5.7,173.0


In [15]:
movie_data['vote_average'].isna().value_counts()

False    45460
True         6
Name: vote_average, dtype: int64

In [16]:
movie_data['vote_count'].fillna(value=movie_data['vote_count'].mean(), inplace=True)
movie_data['vote_count'].isna().value_counts()
movie_data['vote_average'].fillna(value=movie_data['vote_average'].mean(), inplace=True)
movie_data['vote_count'].isna().value_counts()

False    45466
Name: vote_count, dtype: int64

In [17]:
movie_data['production_countries'].isna().value_counts()

False    45463
True         3
Name: production_countries, dtype: int64

In [18]:
movie_data['production_countries'].fillna(method='ffill', inplace=True)
movie_data['production_countries'].isna().value_counts()

False    45466
Name: production_countries, dtype: int64

In [19]:
print(movie_data['production_countries'].iloc[2680][17:19])

US


In [20]:
movie_data['country'] = movie_data['production_countries'].apply(lambda X: X[17:19])
movie_data.drop(columns='production_countries', inplace=True)

In [21]:
movie_data['country'].value_counts()

US    18426
       6286
GB     3070
FR     2705
CA     1498
      ...  
NI        1
ME        1
MQ        1
LA        1
ML        1
Name: country, Length: 144, dtype: int64

In [22]:
movie_data['spoken_languages'].fillna(method='ffill',inplace=True)
movie_data['spoken_languages'].isna().value_counts()

False    45466
Name: spoken_languages, dtype: int64

In [23]:
movie_data['genres'].isna().value_counts()

False    45466
Name: genres, dtype: int64

In [24]:
import re

NameError: name 'genre_list' is not defined

In [74]:
all_genres=[]
def prune_genre(data):
    genre_list=[]
    pattern = 'name\': \'[a-zA-Z]+\''
    found = re.findall(pattern, data)
    if found:
        for i in range(len(found)):
            genre_list.append(found[i][8:-1])
            if found[i][8:-1] not in all_genres:
                all_genres.append(found[i][8:-1])
    else:
        return []
    return genre_list

all_companies = []
def prune_production_companies(data):
    company_list=[]
    pattern ='name\': \'[a-zA-Z\s_]+'
    found = re.findall(pattern, data)
    if found:
        for i in range(len(found)):
            company_list.append(found[i][8:-1])
            if found[i][8:-1] not in all_companies:
                all_companies.append(found[i][8:-1])
    else:
        return []
    return company_list
        

In [75]:
movie_data['genre_list'] = movie_data['genres'].apply(prune_genre)
movie_data['production_companies'].fillna(method='backfill', inplace=True)

In [73]:
coy = movie_data.loc[1,'production_companies']
print(coy)
pattern = 'name\': \'[a-zA-Z\s_]+'
re.findall(pattern, coy)

[{'name': 'TriStar Pictures', 'id': 559}, {'name': 'Teitler Film', 'id': 2550}, {'name': 'Interscope Communications', 'id': 10201}]


["name': 'TriStar Pictures",
 "name': 'Teitler Film",
 "name': 'Interscope Communications"]

In [76]:
movie_data['pro_co'] = movie_data['production_companies'].apply(prune_production_companies)

In [81]:
movie_data[['pro_co','production_companies']].head(10)
movie_data.drop(columns=['production_companies'], inplace=True)

In [28]:
movie_data['genre_list'].head()

0     [Animation, Comedy, Family]
1    [Adventure, Fantasy, Family]
2               [Romance, Comedy]
3        [Comedy, Drama, Romance]
4                        [Comedy]
Name: genre_list, dtype: object

In [82]:
# Extract total genre list from genre_list
len(all_companies)
movie_data.head()
#There are way too many companies, would be ridiculous to convert this to a feature so I'll have to drop it for now

,adult,belongs_to_collection,budget,genres,homepage,id,imdb_id,original_language,original_title,overview,popularity,poster_path,release_date,revenue,runtime,spoken_languages,status,tagline,title,video,vote_average,vote_count,country,genre_list,Animation,Comedy,Family,Adventure,Fantasy,Romance,Drama,Action,Crime,Thriller,Horror,History,Mystery,War,Foreign,Music,Documentary,Western,Aniplex,GoHands,pro_co
0,False,"{'id': 10194, 'name': 'Toy Story Collection', ...",30000000,"[{'id': 16, 'name': 'Animation'}, {'id': 35, '...",http://toystory.disney.com/toy-story,862,tt0114709,en,Toy Story,"Led by Woody, Andy's toys live happily in his ...",21.9469,/rhIRbceoE9lR4veEXuwCC2wARtG.jpg,1995-10-30,373554033.0,81.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,NaN,Toy Story,False,7.7,5415.0,US,"[Animation, Comedy, Family]",1,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,[Pixar Animation Studio]
1,False,NaN,65000000,"[{'id': 12, 'name': 'Adventure'}, {'id': 14, '...",NaN,8844,tt0113497,en,Jumanji,When siblings Judy and Peter discover an encha...,17.0155,/vzmL6fP7aPKNKPRTFnZmiUfciyV.jpg,1995-12-15,262797249.0,104.0,"[{'iso_639_1': 'en', 'name': 'English'}, {'iso...",Released,Roll the dice and unleash the excitement!,Jumanji,False,6.9,2413.0,US,"[Adventure, Fantasy, Family]",0,0,1,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,"[TriStar Picture, Teitler Fil, Interscope Comm..."
2,False,"{'id': 119050, 'name': 'Grumpy Old Men Collect...",0,"[{'id': 10749, 'name': 'Romance'}, {'id': 35, ...",NaN,15602,tt0113228,en,Grumpier Old Men,A family wedding reignites the ancient feud be...,11.7129,/6ksm1sjKMFLbO7UY2i6G1ju9SML.jpg,1995-12-22,0.0,101.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Still Yelling. Still Fighting. Still Ready for...,Grumpier Old Men,False,6.5,92.0,US,"[Romance, Comedy]",0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,"[Warner Bro, Lancaster Gat]"
3,False,NaN,16000000,"[{'id': 35, 'name': 'Comedy'}, {'id': 18, 'nam...",NaN,31357,tt0114885,en,Waiting to Exhale,"Cheated on, mistreated and stepped on, the wom...",3.85949,/16XOMpEaLWkrcPqSQqhTmeJuqQl.jpg,1995-12-22,81452156.0,127.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Friends are the people who let you be yourself...,Waiting to Exhale,False,6.1,34.0,US,"[Comedy, Drama, Romance]",0,1,0,0,0,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,[Twentieth Century Fox Film Corporatio]
4,False,"{'id': 96871, 'name': 'Father of the Bride Col...",0,"[{'id': 35, 'name': 'Comedy'}]",NaN,11862,tt0113041,en,Father of the Bride Part II,Just when George Banks has recovered from his ...,8.38752,/e64sOI48hQXyru7naBFyssKFxVd.jpg,1995-02-10,76578911.0,106.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Just When His World Is Back To Normal... He's ...,Father of the Bride Part II,False,5.7,173.0,US,[Comedy],0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,"[Sandollar Production, Touchstone Picture]"


In [30]:
for genre in all_genres:
    movie_data[genre] = 0

In [31]:
pd.options.display.max_rows = 100
pd.options.display.max_columns = 100

In [32]:
movie_data['id'].nunique()

45436

In [83]:
for i in range(len(movie_data['genre_list'])):
    current_genre_list = movie_data.loc[i,'genre_list']
    for item in current_genre_list:
        movie_data.loc[i,item] = 1



In [84]:
movie_data['adult'].value_counts()

False    45457
True         9
Name: adult, dtype: int64

In [85]:
condition = (movie_data['adult'] != 'True') & (movie_data['adult'] != 'False')
movie_data.loc[condition, 'adult'] = 'False'

In [87]:
movie_data['adult'].value_counts()

False    45457
True         9
Name: adult, dtype: int64

In [88]:
# Generate a more reasonable vote score that accounts for actual number of votes. Borrowing the weighted vote score from IMDB

mean_vote = movie_data['vote_average'].mean()
chosen_quantile = movie_data['vote_count'].quantile(0.85)
print(mean_vote, chosen_quantile)

5.618207215133888 82.0


In [89]:
chosen_data = movie_data.copy().loc[movie_data['vote_count'] >= chosen_quantile]
assert len(chosen_data['vote_count']) == len(chosen_data['vote_average'])
chosen_data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 6838 entries, 0 to 45441
Data columns (total 45 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   adult                  6838 non-null   object 
 1   belongs_to_collection  1661 non-null   object 
 2   budget                 6838 non-null   object 
 3   genres                 6838 non-null   object 
 4   homepage               2331 non-null   object 
 5   id                     6838 non-null   object 
 6   imdb_id                6838 non-null   object 
 7   original_language      6838 non-null   object 
 8   original_title         6838 non-null   object 
 9   overview               6820 non-null   object 
 10  popularity             6833 non-null   object 
 11  poster_path            6835 non-null   object 
 12  release_date           6834 non-null   object 
 13  revenue                6832 non-null   float64
 14  runtime                6832 non-null   float64
 15  spo

In [90]:
# Create a weighted rating using IMDB formula for weighted rating (v/(v+m) * R) + (m/(m+v) * C)
def weighted_rating(data, m=chosen_quantile, C=mean_vote):
    v = data['vote_count']
    R = data['vote_average']
    return (v/(v+m) * R) + (m/(m+v) * C)

In [91]:
chosen_data['score'] = chosen_data.apply(weighted_rating, axis=1)
chosen_data['id'].value_counts()

12600     2
10991     2
15028     2
4912      2
5511      2
         ..
159932    1
41201     1
10498     1
16995     1
10225     1
Name: id, Length: 6831, dtype: int64

In [93]:
print(len(chosen_data))
chosen_data.drop_duplicates(subset=['id'], inplace=True)
print(len(chosen_data))

6831
6831


In [95]:
chosen_data.head()

,adult,belongs_to_collection,budget,genres,homepage,id,imdb_id,original_language,original_title,overview,popularity,poster_path,release_date,revenue,runtime,spoken_languages,status,tagline,title,video,vote_average,vote_count,country,genre_list,Animation,Comedy,Family,Adventure,Fantasy,Romance,Drama,Action,Crime,Thriller,Horror,History,Mystery,War,Foreign,Music,Documentary,Western,Aniplex,GoHands,pro_co,score
0,False,"{'id': 10194, 'name': 'Toy Story Collection', ...",30000000,"[{'id': 16, 'name': 'Animation'}, {'id': 35, '...",http://toystory.disney.com/toy-story,862,tt0114709,en,Toy Story,"Led by Woody, Andy's toys live happily in his ...",21.9469,/rhIRbceoE9lR4veEXuwCC2wARtG.jpg,1995-10-30,373554033.0,81.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,NaN,Toy Story,False,7.7,5415.0,US,"[Animation, Comedy, Family]",1,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,[Pixar Animation Studio],7.668945
1,False,NaN,65000000,"[{'id': 12, 'name': 'Adventure'}, {'id': 14, '...",NaN,8844,tt0113497,en,Jumanji,When siblings Judy and Peter discover an encha...,17.0155,/vzmL6fP7aPKNKPRTFnZmiUfciyV.jpg,1995-12-15,262797249.0,104.0,"[{'iso_639_1': 'en', 'name': 'English'}, {'iso...",Released,Roll the dice and unleash the excitement!,Jumanji,False,6.9,2413.0,US,"[Adventure, Fantasy, Family]",0,0,1,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,"[TriStar Picture, Teitler Fil, Interscope Comm...",6.857873
2,False,"{'id': 119050, 'name': 'Grumpy Old Men Collect...",0,"[{'id': 10749, 'name': 'Romance'}, {'id': 35, ...",NaN,15602,tt0113228,en,Grumpier Old Men,A family wedding reignites the ancient feud be...,11.7129,/6ksm1sjKMFLbO7UY2i6G1ju9SML.jpg,1995-12-22,0.0,101.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Still Yelling. Still Fighting. Still Ready for...,Grumpier Old Men,False,6.5,92.0,US,"[Romance, Comedy]",0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,"[Warner Bro, Lancaster Gat]",6.084442
4,False,"{'id': 96871, 'name': 'Father of the Bride Col...",0,"[{'id': 35, 'name': 'Comedy'}]",NaN,11862,tt0113041,en,Father of the Bride Part II,Just when George Banks has recovered from his ...,8.38752,/e64sOI48hQXyru7naBFyssKFxVd.jpg,1995-02-10,76578911.0,106.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Just When His World Is Back To Normal... He's ...,Father of the Bride Part II,False,5.7,173.0,US,[Comedy],0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,"[Sandollar Production, Touchstone Picture]",5.673698
5,False,NaN,60000000,"[{'id': 28, 'name': 'Action'}, {'id': 80, 'nam...",NaN,949,tt0113277,en,Heat,"Obsessive master thief, Neil McCauley leads a ...",17.9249,/zMyfPUelumio3tiDKPffaUpsQTD.jpg,1995-12-15,187436818.0,170.0,"[{'iso_639_1': 'en', 'name': 'English'}, {'iso...",Released,A Los Angeles Crime Saga,Heat,False,7.7,1886.0,US,"[Action, Crime, Drama, Thriller]",0,0,0,0,0,0,1,1,1,1,0,0,0,0,0,0,0,0,0,0,"[Regency Enterprise, Forward Pas, Warner Bro]",7.613259


In [111]:
columns_to_drop = ['imdb_id','original_title','overview','poster_path','genres','homepage','belongs_to_collection','release_date','spoken_languages','tagline','title']

In [112]:
final_dataset = chosen_data.drop(columns=columns_to_drop)
final_dataset.index= final_dataset['id']

In [113]:
final_dataset.head()

,adult,budget,id,original_language,popularity,revenue,runtime,status,video,vote_average,vote_count,country,genre_list,Animation,Comedy,Family,Adventure,Fantasy,Romance,Drama,Action,Crime,Thriller,Horror,History,Mystery,War,Foreign,Music,Documentary,Western,Aniplex,GoHands,pro_co,score
id,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
862,False,30000000,862,en,21.9469,373554033.0,81.0,Released,False,7.7,5415.0,US,"[Animation, Comedy, Family]",1,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,[Pixar Animation Studio],7.668945
8844,False,65000000,8844,en,17.0155,262797249.0,104.0,Released,False,6.9,2413.0,US,"[Adventure, Fantasy, Family]",0,0,1,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,"[TriStar Picture, Teitler Fil, Interscope Comm...",6.857873
15602,False,0,15602,en,11.7129,0.0,101.0,Released,False,6.5,92.0,US,"[Romance, Comedy]",0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,"[Warner Bro, Lancaster Gat]",6.084442
11862,False,0,11862,en,8.38752,76578911.0,106.0,Released,False,5.7,173.0,US,[Comedy],0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,"[Sandollar Production, Touchstone Picture]",5.673698
949,False,60000000,949,en,17.9249,187436818.0,170.0,Released,False,7.7,1886.0,US,"[Action, Crime, Drama, Thriller]",0,0,0,0,0,0,1,1,1,1,0,0,0,0,0,0,0,0,0,0,"[Regency Enterprise, Forward Pas, Warner Bro]",7.613259


In [109]:
final_dataset['status'].fillna(value='Released', inplace=True)

In [128]:
final_dataset['status'].unique()
final_dataset.drop(columns='id',inplace=True)

In [129]:
def treat_budget(data):
    if data.isnumeric():
        return float(data)
    else:
        print(data)
        return 0.0
final_dataset['budget'] = final_dataset['budget'].apply(treat_budget)
# Just 3 items were set to 0.0 budget

AttributeError: 'float' object has no attribute 'isnumeric'

In [130]:
final_dataset.info()

<class 'pandas.core.frame.DataFrame'>
Index: 6831 entries, 862 to 14008
Data columns (total 34 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   adult              6831 non-null   object 
 1   budget             6831 non-null   float64
 2   original_language  6831 non-null   object 
 3   popularity         6826 non-null   object 
 4   revenue            6825 non-null   float64
 5   runtime            6825 non-null   float64
 6   status             6824 non-null   object 
 7   video              6825 non-null   object 
 8   vote_average       6831 non-null   float64
 9   vote_count         6831 non-null   float64
 10  country            6831 non-null   object 
 11  genre_list         6831 non-null   object 
 12  Animation          6831 non-null   int64  
 13  Comedy             6831 non-null   int64  
 14  Family             6831 non-null   int64  
 15  Adventure          6831 non-null   int64  
 16  Fantasy            6831 no

In [131]:
final_dataset.to_csv('movie_data_training_set.csv')